<a href="https://colab.research.google.com/github/lorenafc/Capita_Selecta_RHI50403/blob/main/authors_encoding_geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import geopandas as gpd
import requests  # To make API requests
import html  # Import the html module for decoding HTML entities
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError
import time
import logging
from tqdm import tqdm # to show the percentage of the geocoding process for each city
from geopandas.tools import geocode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# upload the file
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors (1).xlsx


In [ ]:
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities

##The code below takes too long to run, so I divided it in subsets of 10.000 rows (and still, it took more than 3h to run each subset). After the geocoding, I merged all the files to use it for the map vizualization

In [ ]:
smalldataset_ten_thousand = authors_small_cities_copy.iloc[10000:20000,:] # here I added, for isntance, (....).iloc[10000:20000,:]
print(smalldataset_ten_thousand.shape)

(10000, 10)


#ENCODING FIX (not all of the names were fixed)

In [ ]:


### Function to fix encoding issues #source chatgpt
### THIS FUNCTION FIXED MOST OF THE LOCATIONS, HOWEVER NOT ALL OF THEM.

# import html  # Import the html module for decoding HTML entities


def fix_encoding(text):
    if not isinstance(text, str):
        return text

    # Decode HTML entities first
    text = html.unescape(text)

    # Define possible encodings
    encodings = ['utf-8', 'latin1', 'windows-1252', 'windows-1250', "latin2", "iso-8859-1", "iso-8859-2",'iso-8859-15', 'macroman', 'ascii', 'cp1250', 'cp1251',
        'cp1253', 'cp1254', 'cp1257', 'iso-8859-2', 'iso-8859-3', 'iso-8859-4',
        'iso-8859-5', 'iso-8859-6', 'iso-8859-7',
        'iso-8859-8', 'iso-8859-9', 'koi8-r', 'koi8-u']

    # Try decoding and encoding combinations
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    # Try decoding and then encoding with different encodings
                    decoded_text = text.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    # Check if decoded_text makes sense
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    # Try double encoding scenarios
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    first_pass = text.encode(enc1, errors='replace').decode(enc2, errors='replace')
                    decoded_text = first_pass.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    return text



# Apply the encoding fix function to the columns containing city names
smalldataset_ten_thousand ['borncity'] = smalldataset_ten_thousand['born'].apply(fix_encoding)
smalldataset_ten_thousand ['deathcity'] = smalldataset_ten_thousand ['death'].apply(fix_encoding)
smalldataset_ten_thousand['activecity'] = smalldataset_ten_thousand['active'].apply(fix_encoding)


# Display the first few rows to verify the changes
print(smalldataset_ten_thousand[["born",'borncity', "death", 'deathcity']].head(15))

      born  borncity death deathcity
50000  NaN       NaN   NaN       NaN
50001  NaN       NaN   NaN       NaN
50002  NaN       NaN   NaN       NaN
50003  NaN       NaN   NaN       NaN
50004  NaN       NaN   NaN       NaN
50005  NaN       NaN   NaN       NaN
50006  NaN       NaN   NaN       NaN
50007  NaN       NaN   NaN       NaN
50008  NaN       NaN   NaN       NaN
50009  NaN       NaN   NaN       NaN
50010  NaN       NaN   NaN       NaN
50011  NaN       NaN   NaN       NaN
50012  NaN       NaN   NaN       NaN
50013  NaN       NaN   NaN       NaN
50014  NaN       NaN   NaN       NaN


<ipython-input-25-3e3113cf24b3>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand ['borncity'] = smalldataset_ten_thousand['born'].apply(fix_encoding)
<ipython-input-25-3e3113cf24b3>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand ['deathcity'] = smalldataset_ten_thousand ['death'].apply(fix_encoding)
<ipython-input-25-3e3113cf24b3>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [ ]:
# Cleaning the data

#renaming the columns of the copy
smalldataset_ten_thousand_rename = smalldataset_ten_thousand.rename(columns = {'cleanedbirth':'bitrhyear',
                       'cleaneddeath':'deathyear', "finalname":"nameandbirthdeathyear"},
            inplace = True)
smalldataset_ten_thousand_cleaned = smalldataset_ten_thousand.drop(columns=["originalbirth","originaldeath"])

#change the columns order:
smalldataset_ten_thousand_cleaned=smalldataset_ten_thousand_cleaned[['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'born', 'borncity', 'death', 'deathcity','active', 'activecity']]


<ipython-input-26-b8a4a75dc11a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand_rename = smalldataset_ten_thousand.rename(columns = {'cleanedbirth':'bitrhyear',


#GEOCODING

In [ ]:
# keeping the libraries here too in comments to make it easier to verify the ones used for the geocoding

# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError
# import time
# import logging
# from tqdm import tqdm

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Nominatim geocoder with a user agent
geolocator = Nominatim(user_agent="my_Geocoder_application")

# Function to geocode city names with error handling and rate limiting
def geocode_city(city_name):
    try:
        # Rate limiting
        time.sleep(1)
        # Attempt to geocode the city name with a timeout
        location = geolocator.geocode(city_name, timeout=1)
        if location:
            return (city_name, location.latitude, location.longitude)
        else:
            logging.warning(f"Geocoding failed for: {city_name}")
            return (city_name, None, None)
    except GeopyError as e:
        logging.error(f"Geocoding error for {city_name}: {e}")
        return (city_name, None, None)

# Function to geocode a column of city names and map the results back to the DataFrame
def geocode_column(df, column_name, lat_col_name, lon_col_name):
    # Filter out rows where the column is NaN
    df_filtered = df.dropna(subset=[column_name])

    # Extract unique city names from the DataFrame column
    unique_cities = list(set(df_filtered[column_name]))

    # Geocode unique city names with progress tracking
    geocoded_results = [geocode_city(city) for city in tqdm(unique_cities, desc=f'Geocoding {column_name}')]

    # Create a dictionary to map city names to their geocoded coordinates
    geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results}

    # Map geocoded coordinates back to the original DataFrame
    df[f'{column_name}_coordinates'] = df[column_name].map(geocode_dict)

    # Ensure coordinates are always tuples with two elements
    df[f'{column_name}_coordinates'] = df[f'{column_name}_coordinates'].apply(lambda x: (x if isinstance(x, tuple) else (None, None)))

    # Extract latitude and longitude from coordinates, filling NaN where necessary
    df[[lat_col_name, lon_col_name]] = df[f'{column_name}_coordinates'].apply(pd.Series)

    # Drop the coordinates column
    df.drop(columns=[f'{column_name}_coordinates'], inplace=True)

    return df


# Geocode activecity, borncity, and deathcity columns with distinct names for latitude and longitude
columns_to_geocode = [
    ('borncity', 'latitude_born', 'longitude_born'),
    ('deathcity', 'latitude_death', 'longitude_death'),
    ('activecity', 'latitude_active', 'longitude_active')
]


# Applying the geocoding
for column, lat_col, lon_col in columns_to_geocode:
    smalldataset_ten_thousand_cleaned = geocode_column(smalldataset_ten_thousand_cleaned, column, lat_col, lon_col)

# Display the df with geocoded coordinates
print(smalldataset_ten_thousand_cleaned.head(15))



Geocoding borncity: 0it [00:00, ?it/s]
Geocoding activecity:  13%|█▎        | 630/4831 [11:57<1:24:03,  1.20s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Pierre+de+La+Vergne+de+Tressan&format=json&limit=1
ERROR:root:Geocoding error for Pierre de La Vergne de Tressan: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Pierre+de+La+Vergne+de+Tressan&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Geocoding activecity:  56%|█████▋    | 2725/4831 [52:08<40:42,  1.16s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'R

In [ ]:
smalldataset_ten_thousand_cleaned.drop(['born', 'death', "active"], axis=1)

In [ ]:
# export the result as excel file
file_name = '2_smalldataset.xlsx'
smalldataset_ten_thousand_cleaned.to_excel(file_name)

In [ ]:
# if you want also to save in Google Drive uncomment below

# # Specify the path in Google Drive
# file_path = '/content/drive/My Drive/driv_author_50k_-60k_active_bor_death_geocoded.xlsx'

# # Save also the df to an Excel file in Google Drive
# smalldataset_ten_thousand_cleaned.to_excel(file_path)


## Merging the excel files

In [ ]:
# If you made subsets to geocode the whole data, uncomment below and adjust according to your data names


# base_path = '/content/drive/My Drive/'  # location of the geocoded excel files

# # List of file names
# file_names = [
#     '1_smalldataset.xlsx',
#     '2_smalldataset.xlsx',
#     '3_smalldataset.xlsx',
#     '4_smalldataset.xlsx',
#     '5_smalldataset.xlsx',
#     '6_smalldataset.xlsx',
#     '7_smalldataset.xlsx',
#     '8_smalldataset.xlsx'
# ]

# # Read each file into a DataFrame and store them in a list
# dataframes = [pd.read_excel(file_name) for file_name in file_names]

# # Concatenate all DataFrames into one, ignoring the original index
# merged_dataframe = pd.concat(dataframes, ignore_index=True)

# # Save the merged DataFrame to a new Excel file in your Google Drive
# merged_dataframe.to_excel(base_path + 'merged_additional_authors_encoded.xlsx', index=False)

